# Setup

When using pandas (and numpy), normally one imports the libraries as follows:
```python
import pandas as pd
import numpy as np
```

If you want to plot - that's another set of libraries:
```python
from matplotlib import pyplot as plt
%matplotlib inline
import numpy as np
```

**piper** is a suite of python modules built on top of pandas that complement existing functions :).<br>
Entering a single 'magic' command, makes available pandas, numpy, plotting libraries and piper helper functions into the current notebook.  
<ul>
<li> Records last time the notebook was run 
<li> allows you to externally parameterise the notebook (run the same notebook with different params).
</ul>

In [1]:
from piper import piper
from piper.defaults import *
from piper.io import *
from piper.verbs import *
from piper.configure import get_config

piper version 0.0.7, last run: Friday, 26 February 2021 14:54:49


## default folders and config details

**piper** gives you access to a json formatted configuration file (config.json).<p>
Here, you can store default folder values, define external variables used in the notebook etc.

In [2]:
get_config('config.json')

{'meta': {'project': 'notebook project'},
 'folders': ['inputs', 'src', 'sql', 'docs', 'outputs', 'reports'],
 'variables': {'dummy_variable': 'dummy'},
 'connections': {'location': 'connections.json'},
 'excel': {'meta': 'xl_meta.json', 'formats': 'formats.json'},
 'mail_config': {'server': 'localhost',
  'port': 25,
  'sender': 'someone@acme.com'},
 'project': 'notebook project'}

See below, we can read through the config.json file and print out all the values easily.<p>
Note the use of **list** and **dictionary** comprehensions to explode respective details. **NEAT!!**

In [3]:
from piper.configure import get_config
config = get_config('config.json')

In [4]:
for k, v in config.items():
    logger.info(f'{k}')
    if isinstance(v, str):
        logger.info('\tvalue: ' + v)
    elif isinstance(v, list):
        [logger.info('\t value: ' + value) for value in v]
    elif isinstance(v, dict):
        {logger.info(f'\tkey: {k} -> values: {v}') for k, v in v.items()}

meta
	key: project -> values: notebook project
folders
	 value: inputs
	 value: src
	 value: sql
	 value: docs
	 value: outputs
	 value: reports
variables
	key: dummy_variable -> values: dummy
connections
	key: location -> values: connections.json
excel
	key: meta -> values: xl_meta.json
	key: formats -> values: formats.json
mail_config
	key: server -> values: localhost
	key: port -> values: 25
	key: sender -> values: someone@acme.com
project
	value: notebook project


# DataFrame - Transformations

A typical issue with data is that it is not 'clean'. Transformations are required to get data into the right format  for loading or passing on to another system. Let's **read excel** data into a dataframe using standard pandas **read_excel** command. The dataframe has a lot of issues:<ol>
<li> bad column names
<li> names and description in columns 3 and 4 below need reformatting.
<li> invalid price and qty values
<li> dates not in correct format for use in Excel (for example).
</ol>

In [5]:
# Reading in the file is as simple as passing file name to read_excel function
df_original = pd.read_excel("inputs/Test excel workbook.xlsx")

# Making a copy of the dataframe (good practice, means you can 'reload' without going back to file.)
df = df_original.copy(deep=True)

# pandas .head() function defaults to showing first 5 rows in dataframe
df.head()

,Gropuing cde_,Order_NBR,This column name is too long,Second column,Quantity,Price,Effective,Expired,TranSACTion DATE,A delimitted LIsT
0,A100,23899001,First row,"Scally, Aidan",14,"1,23",21.10.2015,31.12.2019,20.08.2018,123;456;789
1,A101,23899002,SECOnd Row,"McAllister, Eoin",103,"4,32",21.10.2016,31.12.2020,20.08.2017,ss;11;33
2,A101,23899003,Thrid Row,"Tarpey, Mike",1,"3 4,32",21.10.2017,31.12.2021,20.08.2020,A;b;C
3,A102,23899004,fOuRth ROW,"Denton, Alan",13,49,21.10.2018,31.12.2022,20.08.2021,cat;dog;books
4,A103,23899005,fIFTH rOw,"Dallis, Theo",19,"45.7,98",21.10.2019,31.12.2023,20.08.2022,x;y;z


## Rename columns

Renaming columns can be done in two ways depending on what you want to do:
<ol>
<li> rename ALL columns in one go.
<li> rename selected columns
<ol>

In [6]:
columns_renamed = ['grouping', 'orders', 'description', 'second_col',
                   'qty', 'price', 'effective', 'expired', 'date', 'delimitted_list']

# df.columns is an 'attribute' or property of the dataframe and can be accessed/updated directly
df.columns = columns_renamed

# dfxl head function gives us same functionality as pandas .head() but with row, col count information.
head(df)

12 rows, 10 columns


,grouping,orders,description,second_col,qty,price,effective,expired,date,delimitted_list
0,A100,23899001,First row,"Scally, Aidan",14,"1,23",21.10.2015,31.12.2019,20.08.2018,123;456;789
1,A101,23899002,SECOnd Row,"McAllister, Eoin",103,"4,32",21.10.2016,31.12.2020,20.08.2017,ss;11;33
2,A101,23899003,Thrid Row,"Tarpey, Mike",1,"3 4,32",21.10.2017,31.12.2021,20.08.2020,A;b;C
3,A102,23899004,fOuRth ROW,"Denton, Alan",13,49,21.10.2018,31.12.2022,20.08.2021,cat;dog;books


## Chaining functions - clean strings example

In example below, the description field contains embedded blanks.  How can you clean this to remove all the addtional spaces before, after and in between each of the words? Impossible?

In [7]:
df.description #.to_frame()

0     First                        row
1                         SECOnd   Row
2                     Thrid        Row
3                    fOuRth        ROW
4                            fIFTH rOw
5                     sIxTH        rOw
6                          SeVENtH roW
7                           EIghTh RoW
8                       NINTH      row
9                  TEnTh           rOW
10                     fOuRth      Row
11                      fIFTH      rOw
Name: description, dtype: object

pandas has a massive list of 'string' functions... below each function is 'chained' one after the other - lowecasing, stripping blanks (before and after words), replacing 'ri' with 'ir' and finally stripping and capturing the words (\w) using regular expressions (regex) and reversing the word order.

In [8]:
df['description'] = (df['description'].str.lower()
                                      .str.strip()
                                      .str.title()
                                      .str.replace('ri','ir', regex=True)
                                      .str.replace(r'(\w)\s+(\w)', r'\1 \2', regex=True))
df.description #.to_frame()

0       First Row
1      Second Row
2       Third Row
3      Fourth Row
4       Fifth Row
5       Sixth Row
6     Seventh Row
7      Eighth Row
8       Ninth Row
9       Tenth Row
10     Fourth Row
11      Fifth Row
Name: description, dtype: object

Another example, we want to strip blanks, capitalize each word and reverse the word sequence.

In [9]:
df.second_col #.to_frame()

0                     Scally, Aidan
1                  McAllister, Eoin
2                      Tarpey, Mike
3               Denton,        Alan
4     Dallis,                  Theo
5                       HUNT, DerEK
6                 Goddard,     TONY
7              Whitaker,    Matthew
8                 Seiffert, CARsteN
9               Freer,        Craig
10                   DENTON,   Alan
11           Dallis,           THEO
Name: second_col, dtype: object

In [10]:
# Voila! Simple, chained functions

df.second_col = (df.second_col.str.strip()
                              .str.title()
                              .str.replace(r'(\w+),\s+(\w+)', r'\2 \1', regex=True))
df.second_col #.to_frame()

0         Aidan Scally
1      Eoin Mcallister
2          Mike Tarpey
3          Alan Denton
4          Theo Dallis
5           Derek Hunt
6         Tony Goddard
7     Matthew Whitaker
8     Carsten Seiffert
9          Craig Freer
10         Alan Denton
11         Theo Dallis
Name: second_col, dtype: object

## Clean-up

### Numbers

In [11]:
# Again, someone has really messed up here ;) ...
# how can we remove invalid characters to clean up these number values?

df[['qty', 'price']]

,qty,price
0,14,"1,23"
1,103,"4,32"
2,1,"3 4,32"
3,13,49
4,19,"45.7,98"
5,"5---32,14","634,23"
6,178.3035,27.04502
7,"4-2-4,00","56,3.00"
8,"2-4,00","9,.8.00"
9,"18,00.22",563


In [12]:
df.loc[df.qty.str.match(r'[\w]') == True, 'qty'] = (df.qty.str.strip()
                                                          .str.replace(r'[\s\-]', '', regex=True)
                                                          .str.replace(',(\d{2})$',r'.\1', regex=True)
                                                          .str.replace(r',', '', regex=True))

df.loc[df.price.str.match(r'[\w]') == True, 'price'] = (df.price.str.replace(r'\s+', r'', regex=True))

df[['qty', 'price']]

,qty,price
0,14,"1,23"
1,103,"4,32"
2,1,"34,32"
3,13,49
4,19,"45.7,98"
5,532.14,"634,23"
6,178.3035,27.04502
7,424.00,"56,3.00"
8,24.00,"9,.8.00"
9,1800.22,563


In [13]:
# If not alphanumeric, replace any comma with a decimal point.
df.loc[df.price.str.match(r'[\W]') == True, 'price'] = (df.price.str.replace(r',', r'.', regex=True))

# if alphanumeric, replace comma with point, strip spaces and if more than one decimal point, remove 
df.loc[df.price.str.match(r'[\w]') == True, 'price'] = (df.price.str.replace(r',', r'.', regex=True)
                                                                .str.replace(r'\s+', r'', regex=True)
                                                                .str.replace(r'\.{2}',r'', regex=True))
# Check again alphanumeric fields for multiple d.p. also if decimal points found 'between', number - remove ALL points.
df.loc[df.price.str.match(r'[\w]') == True, 'price'] = (df.price.str.replace(r'\.{2}',r'', regex=True)
                                                                .str.replace(r'(\.\d)\.',r'\1', regex=True))
df.price.to_frame()

,price
0,1.23
1,4.32
2,34.32
3,49
4,45.798
5,634.23
6,27.04502
7,56.300
8,98.00
9,563


In [14]:
# Finally, round price to two decimal places - set qty to integer data type.

df.price = df.price.astype(float).round(2)
df.qty = pd.to_numeric(df.qty).astype(int)
df[['price', 'qty']]

,price,qty
0,1.23,14
1,4.32,103
2,34.32,1
3,49.00,13
4,45.80,19
5,634.23,532
6,27.05,178
7,56.30,424
8,98.00,24
9,563.00,1800


### Dates

In [15]:
# NOTE: dfxl allows you to specify number of rows to display # help(head) for details
head(df, 2)
# help(head)

12 rows, 10 columns


,grouping,orders,description,second_col,qty,price,effective,expired,date,delimitted_list
0,A100,23899001,First Row,Aidan Scally,14,1.23,21.10.2015,31.12.2019,20.08.2018,123;456;789
1,A101,23899002,Second Row,Eoin Mcallister,103,4.32,21.10.2016,31.12.2020,20.08.2017,ss;11;33


In [16]:
# Pandas to_datetime guesses very well input format, if that does not work - you can specify exact input format.
df.effective = pd.to_datetime(df.effective)
df.expired = pd.to_datetime(df.expired)

# calculating days duration 
df['duration'] = (df.expired - df.effective).dt.days

df.drop(columns=['date'], inplace=True)
df.sort_values(by='expired', inplace=True)

# Moving a column using combination of insert and pop functions
df.insert(8, 'duration', df.pop('duration'))

head(df, 2)

12 rows, 10 columns


,grouping,orders,description,second_col,qty,price,effective,expired,duration,delimitted_list
0,A100,23899001,First Row,Aidan Scally,14,1.23,2015-10-21,2019-12-31,1532,123;456;789
5,A103,23899006,Sixth Row,Derek Hunt,532,634.23,2019-04-21,2020-05-07,382,alpha; bravo; charlie


### Delimitted data expanded to separate columns

See below, a column 'delimitted_list' with semi-colon (';') separator is easily expanded to three columns 

In [17]:
expanded_list = df.delimitted_list.str.split(';', n=2, expand=True)
expanded_list.columns = ['list_1', 'list_2', 'list_3']
df = pd.concat([df, expanded_list], axis=1)
df.loc[:, 'delimitted_list': 'list_3'].head()

,delimitted_list,list_1,list_2,list_3
0,123;456;789,123,456,789
5,alpha; bravo; charlie,alpha,bravo,charlie
1,ss;11;33,ss,11,33
2,A;b;C,A,b,C
3,cat;dog;books,cat,dog,books


# DataFrame - exploratory data analysis

## head()

In [18]:
head(df, 2)

12 rows, 13 columns


,grouping,orders,description,second_col,qty,price,effective,expired,duration,delimitted_list,list_1,list_2,list_3
0,A100,23899001,First Row,Aidan Scally,14,1.23,2015-10-21,2019-12-31,1532,123;456;789,123,456,789
5,A103,23899006,Sixth Row,Derek Hunt,532,634.23,2019-04-21,2020-05-07,382,alpha; bravo; charlie,alpha,bravo,charlie


## tail()

In [19]:
tail(df, 2)

12 rows, 13 columns


,grouping,orders,description,second_col,qty,price,effective,expired,duration,delimitted_list,list_1,list_2,list_3
7,A104,23899008,Eighth Row,Matthew Whitaker,424,56.3,2019-10-21,2025-12-31,2263,cc;bb;aa,cc,bb,aa
9,A105,23899010,Tenth Row,Craig Freer,1800,563.0,2019-10-21,2025-12-31,2263,40a3;64g;023;,40a3,64g,023;


## info()

In [20]:
# Introduce 'null' values into dataframe
df.loc[df.duration == 2263, 'duration'] = np.NaN

# df.info() # standard pandas
info(df, include_dupes=True, filter_na_cols=False) # dfxl

Dataframe with (rows, cols) (12, 13) consumes 0.01 Mb


,columns,data_type,isna,isnull,unique,dupes,total_count
0,grouping,object,0,0,6,10,12
1,orders,int64,0,0,12,0,12
2,description,object,0,0,10,4,12
3,second_col,object,0,0,10,4,12
4,qty,int64,0,0,10,4,12
5,price,float64,0,0,10,4,12
6,effective,datetime64[ns],0,0,7,7,12
7,expired,datetime64[ns],0,0,8,7,12
8,duration,float64,3,3,3,10,12
9,delimitted_list,object,0,0,12,0,12


## value_counts() vs count()

In [21]:
df.duration.value_counts(normalize=False, dropna=False) # pandas
# df.duration.value_counts(normalize=True, dropna=False).round(2)

1532.0    7
NaN       3
2760.0    1
382.0     1
Name: duration, dtype: int64

In [22]:
count(df, 'duration', add_total=True) 

,index,n
0,1532.0,7
1,NaN,3
2,2760.0,1
3,382.0,1
4,Total,12


In [23]:
df.effective.value_counts()

2019-10-21    5
2018-10-21    2
2017-10-21    1
2015-10-21    1
2018-05-21    1
2016-10-21    1
2019-04-21    1
Name: effective, dtype: int64

In [24]:
count(df, 'effective', add_total=True, percent=True, round=7)

,index,n,%
0,2019-10-21 00:00:00,5,41.666667
1,2018-10-21 00:00:00,2,16.666667
2,2017-10-21 00:00:00,1,8.333333
3,2015-10-21 00:00:00,1,8.333333
4,2018-05-21 00:00:00,1,8.333333
5,2016-10-21 00:00:00,1,8.333333
6,2019-04-21 00:00:00,1,8.333333
7,Total,12,200.000000


## columns()

In [25]:
# piper 'columns()' function is useful for selecting and manipulating dataframe column values.
# It can return multiple objects back (lists, dictionaries, dataframes etc. see help(columns) for details)
columns(df)

['grouping',
 'orders',
 'description',
 'second_col',
 'qty',
 'price',
 'effective',
 'expired',
 'duration',
 'delimitted_list',
 'list_1',
 'list_2',
 'list_3']

In [26]:
# list columns starting with 'e' and return a dictionary (useful for selective column rename)
columns(df, 'e', astype='dict')

{'effective': 'effective', 'expired': 'expired'}

In [27]:
# Combining functions together is easy ;) 

# -> show me columns contains 'list' 
#                 or contains 'e' or 'qty' or ends in 'ers'
head(df[columns(df, 'list|e|qty|.+ers$')], 2)

12 rows, 7 columns


,orders,qty,effective,expired,list_1,list_2,list_3
0,23899001,14,2015-10-21,2019-12-31,123,456,789
5,23899006,532,2019-04-21,2020-05-07,alpha,bravo,charlie


In [28]:
# show me columns ending in letter 'e'
df[columns(df, '\w+e$')].head()

,price,effective
0,1.23,2015-10-21
5,634.23,2019-04-21
1,4.32,2016-10-21
2,34.32,2017-10-21
3,49.00,2018-10-21


## duplicated() - Check for duplicate data

In [29]:
# How can I quickly find duplicate values (for any combination of columns) in a dataframe??

In [30]:
subset = ['description', 'second_col']

df2 = (duplicated(df.copy(deep=True), subset=subset, keep='first').reset_index(drop=True))
head(df2, 6)

12 rows, 14 columns


,duplicate,grouping,orders,description,second_col,qty,price,effective,expired,duration,delimitted_list,list_1,list_2,list_3
0,False,A104,23899008,Eighth Row,Matthew Whitaker,424,56.30,2019-10-21,2025-12-31,NaN,cc;bb;aa,cc,bb,aa
1,False,A103,23899005,Fifth Row,Theo Dallis,19,45.80,2019-10-21,2023-12-31,1532.0,x;y;z,x,y,z
2,True,A103,23899012,Fifth Row,Theo Dallis,19,45.80,2019-10-21,2023-12-31,1532.0,5555;4444;3333,5555,4444,3333
3,False,A100,23899001,First Row,Aidan Scally,14,1.23,2015-10-21,2019-12-31,1532.0,123;456;789,123,456,789
4,False,A102,23899004,Fourth Row,Alan Denton,13,49.00,2018-10-21,2022-12-31,1532.0,cat;dog;books,cat,dog,books
5,True,A102,23899011,Fourth Row,Alan Denton,13,49.00,2018-10-21,2022-12-31,1532.0,d20;tg54;kde40,d20,tg54,kde40


In [31]:
subset = ['price']

(head(duplicated(df2, subset=subset, loc='first', sort=True,
                 column='duplicate price')
      .reset_index(drop=True)
      .drop(columns=['duplicate']), 8))

12 rows, 14 columns


,duplicate price,grouping,orders,description,second_col,qty,price,effective,expired,duration,delimitted_list,list_1,list_2,list_3
0,False,A100,23899001,First Row,Aidan Scally,14,1.23,2015-10-21,2019-12-31,1532.0,123;456;789,123,456,789
1,False,A101,23899002,Second Row,Eoin Mcallister,103,4.32,2016-10-21,2020-12-31,1532.0,ss;11;33,ss,11,33
2,False,A103,23899007,Seventh Row,Tony Goddard,178,27.05,2019-10-21,2025-12-31,NaN,24th May; 18th July; 5th August,24th May,18th July,5th August
3,False,A101,23899003,Third Row,Mike Tarpey,1,34.32,2017-10-21,2021-12-31,1532.0,A;b;C,A,b,C
4,True,A103,23899005,Fifth Row,Theo Dallis,19,45.80,2019-10-21,2023-12-31,1532.0,x;y;z,x,y,z
5,True,A103,23899012,Fifth Row,Theo Dallis,19,45.80,2019-10-21,2023-12-31,1532.0,5555;4444;3333,5555,4444,3333
6,True,A102,23899004,Fourth Row,Alan Denton,13,49.00,2018-10-21,2022-12-31,1532.0,cat;dog;books,cat,dog,books
7,True,A102,23899011,Fourth Row,Alan Denton,13,49.00,2018-10-21,2022-12-31,1532.0,d20;tg54;kde40,d20,tg54,kde40


In [32]:
subset=['description']

(head(duplicated(df2, subset=subset,column='duplicate description')
      .reset_index(drop=True)
      .drop(columns=['duplicate']), 3))

12 rows, 14 columns


,duplicate description,grouping,orders,description,second_col,qty,price,effective,expired,duration,delimitted_list,list_1,list_2,list_3
0,False,A104,23899008,Eighth Row,Matthew Whitaker,424,56.3,2019-10-21,2025-12-31,NaN,cc;bb;aa,cc,bb,aa
1,True,A103,23899005,Fifth Row,Theo Dallis,19,45.8,2019-10-21,2023-12-31,1532.0,x;y;z,x,y,z
2,True,A103,23899012,Fifth Row,Theo Dallis,19,45.8,2019-10-21,2023-12-31,1532.0,5555;4444;3333,5555,4444,3333


## Combine and explode column data

### combining columns into a single 'list' column

In [33]:
df[['list_1', 'list_2', 'list_3']]

,list_1,list_2,list_3
0,123,456,789
5,alpha,bravo,charlie
1,ss,11,33
2,A,b,C
3,cat,dog,books
10,d20,tg54,kde40
4,x,y,z
11,5555,4444,3333
8,1304,53-02,6932
6,24th May,18th July,5th August


In [34]:
# Step 1: Combine columns in to a single Series consisting of three columns values as a list
# then converted to a Series object.
# del df['combined_listcol']
combined_listcol = pd.Series(df[['list_1', 'list_2', 'list_3']].values.tolist())
combined_listcol

# Step 2: Insert series at position 13 (end) of dataframe
df.insert(13, 'combined_listcol', combined_listcol)
head(df.loc[: ,'list_1': 'combined_listcol'])

12 rows, 4 columns


,list_1,list_2,list_3,combined_listcol
0,123,456,789,"[123, 456, 789]"
5,alpha,bravo,charlie,"[d20, tg54, kde40]"
1,ss,11,33,"[alpha, bravo, charlie]"
2,A,b,C,"[ss, 11, 33]"


### explode (extract) values from a single 'list' column

In [35]:
df[['second_col', 'combined_listcol']].head(3)

,second_col,combined_listcol
0,Aidan Scally,"[123, 456, 789]"
5,Derek Hunt,"[d20, tg54, kde40]"
1,Eoin Mcallister,"[alpha, bravo, charlie]"


In [36]:
head(df[['second_col', 'combined_listcol']].explode('combined_listcol'), 4)

36 rows, 2 columns


,second_col,combined_listcol
0,Aidan Scally,123
0,Aidan Scally,456
0,Aidan Scally,789
5,Derek Hunt,d20


## add_formula - Add Excel formula(s)

In [37]:
# del df['X8']
formula = '=CONCATENATE(A{row}, B{row}, C{row})' # See help(add_formula) for details
add_formula(df, column_name='X8', formula=formula, inplace=True)
head(df.loc[:, 'duration': 'X8'], 4)

12 rows, 7 columns


,duration,delimitted_list,list_1,list_2,list_3,combined_listcol,X8
0,1532.0,123;456;789,123,456,789,"[123, 456, 789]","=CONCATENATE(A2, B2, C2)"
5,382.0,alpha; bravo; charlie,alpha,bravo,charlie,"[d20, tg54, kde40]","=CONCATENATE(A3, B3, C3)"
1,1532.0,ss;11;33,ss,11,33,"[alpha, bravo, charlie]","=CONCATENATE(A4, B4, C4)"
2,1532.0,A;b;C,A,b,C,"[ss, 11, 33]","=CONCATENATE(A5, B5, C5)"


In [38]:
# help(add_formula)

# Categorical data

In [39]:
# We want to 'pivot' and summarise the records below, counting how many groups relate to description column
df[['description', 'grouping']].head(8)

,description,grouping
0,First Row,A100
5,Sixth Row,A103
1,Second Row,A101
2,Third Row,A101
3,Fourth Row,A102
10,Fourth Row,A102
4,Fifth Row,A103
11,Fifth Row,A103


In [40]:
df.pivot_table(index=['grouping'],
               columns='description',
               values='orders',
               fill_value=0,
               aggfunc=np.size)

description,Eighth Row,Fifth Row,First Row,Fourth Row,Ninth Row,Second Row,Seventh Row,Sixth Row,Tenth Row,Third Row
grouping,,,,,,,,,,
A100,0,0,1,0,0,0,0,0,0,0
A101,0,0,0,0,0,1,0,0,0,1
A102,0,0,0,2,0,0,0,0,0,0
A103,0,2,0,0,0,0,1,1,0,0
A104,1,0,0,0,0,0,0,0,0,0
A105,0,0,0,0,1,0,0,0,1,0


In [41]:
# Required 'sequence' for categories
categories = [
    'First Row', 'Second Row', 'Third Row', 'Fourth Row', 'Fifth Row',
    'Sixth Row', 'Seventh Row', 'Eighth Row', 'Ninth Row', 'Tenth Row'
]
df.description = pd.Categorical(values=df.description,
                                categories=categories,
                                ordered=True)

df_categorical = df.copy(deep=True)
df_categorical.pivot_table(index=['grouping'],
                           columns='description',
                           values='orders',
                           fill_value=0,
                           aggfunc=np.size)

description,First Row,Second Row,Third Row,Fourth Row,Fifth Row,Sixth Row,Seventh Row,Eighth Row,Ninth Row,Tenth Row
grouping,,,,,,,,,,
A100,1,0,0,0,0,0,0,0,0,0
A101,0,1,1,0,0,0,0,0,0,0
A102,0,0,0,2,0,0,0,0,0,0
A103,0,0,0,0,2,1,1,0,0,0
A104,0,0,0,0,0,0,0,1,0,0
A105,0,0,0,0,0,0,0,0,1,1


# DataFrame - Select/filter and extract

## Simple filter

In [42]:
df[df.second_col == 'Theo Dallis']

,grouping,orders,description,second_col,qty,price,effective,expired,duration,delimitted_list,list_1,list_2,list_3,combined_listcol,X8
4,A103,23899005,Fifth Row,Theo Dallis,19,45.8,2019-10-21,2023-12-31,1532.0,x;y;z,x,y,z,"[cat, dog, books]","=CONCATENATE(A8, B8, C8)"
11,A103,23899012,Fifth Row,Theo Dallis,19,45.8,2019-10-21,2023-12-31,1532.0,5555;4444;3333,5555,4444,3333,"[40a3, 64g, 023;]","=CONCATENATE(A9, B9, C9)"


## Multiple filter - AND

In [43]:
filter_1 = df.grouping == 'A103'
filter_2 = df.second_col == 'Theo Dallis'

df[filter_1 & filter_2]

,grouping,orders,description,second_col,qty,price,effective,expired,duration,delimitted_list,list_1,list_2,list_3,combined_listcol,X8
4,A103,23899005,Fifth Row,Theo Dallis,19,45.8,2019-10-21,2023-12-31,1532.0,x;y;z,x,y,z,"[cat, dog, books]","=CONCATENATE(A8, B8, C8)"
11,A103,23899012,Fifth Row,Theo Dallis,19,45.8,2019-10-21,2023-12-31,1532.0,5555;4444;3333,5555,4444,3333,"[40a3, 64g, 023;]","=CONCATENATE(A9, B9, C9)"


## Multiple filter - OR

In [44]:
filter_1 = df.grouping == 'A103'
filter_2 = df.second_col == 'Theo Dallis'

df[filter_1 | filter_2]

,grouping,orders,description,second_col,qty,price,effective,expired,duration,delimitted_list,list_1,list_2,list_3,combined_listcol,X8
5,A103,23899006,Sixth Row,Derek Hunt,532,634.23,2019-04-21,2020-05-07,382.0,alpha; bravo; charlie,alpha,bravo,charlie,"[d20, tg54, kde40]","=CONCATENATE(A3, B3, C3)"
4,A103,23899005,Fifth Row,Theo Dallis,19,45.80,2019-10-21,2023-12-31,1532.0,x;y;z,x,y,z,"[cat, dog, books]","=CONCATENATE(A8, B8, C8)"
11,A103,23899012,Fifth Row,Theo Dallis,19,45.80,2019-10-21,2023-12-31,1532.0,5555;4444;3333,5555,4444,3333,"[40a3, 64g, 023;]","=CONCATENATE(A9, B9, C9)"
6,A103,23899007,Seventh Row,Tony Goddard,178,27.05,2019-10-21,2025-12-31,NaN,24th May; 18th July; 5th August,24th May,18th July,5th August,"[x, y, z]","=CONCATENATE(A11, B11, C11)"


## str.extractall()

In [45]:
df.second_col.str.extractall('(Theo|Aidan) (\w+)', flags=re.I)

,,0,1
,match,,
0,0,Aidan,Scally
4,0,Theo,Dallis
11,0,Theo,Dallis


## str.findall()

In [46]:
df.description.str.findall('fourth', flags=re.I)

0           []
5           []
1           []
2           []
3     [Fourth]
10    [Fourth]
4           []
11          []
8           []
6           []
7           []
9           []
Name: description, dtype: object

## str.match()

In [47]:
df[df.delimitted_list.str.match('55')]

,grouping,orders,description,second_col,qty,price,effective,expired,duration,delimitted_list,list_1,list_2,list_3,combined_listcol,X8
11,A103,23899012,Fifth Row,Theo Dallis,19,45.8,2019-10-21,2023-12-31,1532.0,5555;4444;3333,5555,4444,3333,"[40a3, 64g, 023;]","=CONCATENATE(A9, B9, C9)"


In [48]:
df[df.description.str.match('(first|fifth)', flags=re.I)]

,grouping,orders,description,second_col,qty,price,effective,expired,duration,delimitted_list,list_1,list_2,list_3,combined_listcol,X8
0,A100,23899001,First Row,Aidan Scally,14,1.23,2015-10-21,2019-12-31,1532.0,123;456;789,123,456,789,"[123, 456, 789]","=CONCATENATE(A2, B2, C2)"
4,A103,23899005,Fifth Row,Theo Dallis,19,45.80,2019-10-21,2023-12-31,1532.0,x;y;z,x,y,z,"[cat, dog, books]","=CONCATENATE(A8, B8, C8)"
11,A103,23899012,Fifth Row,Theo Dallis,19,45.80,2019-10-21,2023-12-31,1532.0,5555;4444;3333,5555,4444,3333,"[40a3, 64g, 023;]","=CONCATENATE(A9, B9, C9)"


## str.contains()

In [49]:
df[df.second_col.str.contains('Tony|Derek|Theo')]

,grouping,orders,description,second_col,qty,price,effective,expired,duration,delimitted_list,list_1,list_2,list_3,combined_listcol,X8
5,A103,23899006,Sixth Row,Derek Hunt,532,634.23,2019-04-21,2020-05-07,382.0,alpha; bravo; charlie,alpha,bravo,charlie,"[d20, tg54, kde40]","=CONCATENATE(A3, B3, C3)"
4,A103,23899005,Fifth Row,Theo Dallis,19,45.80,2019-10-21,2023-12-31,1532.0,x;y;z,x,y,z,"[cat, dog, books]","=CONCATENATE(A8, B8, C8)"
11,A103,23899012,Fifth Row,Theo Dallis,19,45.80,2019-10-21,2023-12-31,1532.0,5555;4444;3333,5555,4444,3333,"[40a3, 64g, 023;]","=CONCATENATE(A9, B9, C9)"
6,A103,23899007,Seventh Row,Tony Goddard,178,27.05,2019-10-21,2025-12-31,NaN,24th May; 18th July; 5th August,24th May,18th July,5th August,"[x, y, z]","=CONCATENATE(A11, B11, C11)"


## df.query()

### single selection

In [50]:
query = """ grouping == 'A100' """
df.query(query)

,grouping,orders,description,second_col,qty,price,effective,expired,duration,delimitted_list,list_1,list_2,list_3,combined_listcol,X8
0,A100,23899001,First Row,Aidan Scally,14,1.23,2015-10-21,2019-12-31,1532.0,123;456;789,123,456,789,"[123, 456, 789]","=CONCATENATE(A2, B2, C2)"


### multiple OR

In [51]:
query = """(grouping == 'A101') or\
           (second_col.str.contains('Mike Tarpey')) or\
           (orders < 23899004) """
df.query(query)

,grouping,orders,description,second_col,qty,price,effective,expired,duration,delimitted_list,list_1,list_2,list_3,combined_listcol,X8
0,A100,23899001,First Row,Aidan Scally,14,1.23,2015-10-21,2019-12-31,1532.0,123;456;789,123,456,789,"[123, 456, 789]","=CONCATENATE(A2, B2, C2)"
1,A101,23899002,Second Row,Eoin Mcallister,103,4.32,2016-10-21,2020-12-31,1532.0,ss;11;33,ss,11,33,"[alpha, bravo, charlie]","=CONCATENATE(A4, B4, C4)"
2,A101,23899003,Third Row,Mike Tarpey,1,34.32,2017-10-21,2021-12-31,1532.0,A;b;C,A,b,C,"[ss, 11, 33]","=CONCATENATE(A5, B5, C5)"


In [52]:
query = """(grouping == 'A101') |\
           (second_col.str.contains('Mike Tarpey')) |\
           (orders < 23899004) """
df.query(query)

,grouping,orders,description,second_col,qty,price,effective,expired,duration,delimitted_list,list_1,list_2,list_3,combined_listcol,X8
0,A100,23899001,First Row,Aidan Scally,14,1.23,2015-10-21,2019-12-31,1532.0,123;456;789,123,456,789,"[123, 456, 789]","=CONCATENATE(A2, B2, C2)"
1,A101,23899002,Second Row,Eoin Mcallister,103,4.32,2016-10-21,2020-12-31,1532.0,ss;11;33,ss,11,33,"[alpha, bravo, charlie]","=CONCATENATE(A4, B4, C4)"
2,A101,23899003,Third Row,Mike Tarpey,1,34.32,2017-10-21,2021-12-31,1532.0,A;b;C,A,b,C,"[ss, 11, 33]","=CONCATENATE(A5, B5, C5)"


### multiple AND / OR with substitutional variables

In [53]:
list_of_groups = ['A100', 'A101', 'A103']
duration_criteria = 380

query = """(grouping in @list_of_groups) and\
           (~second_col.str.startswith('Theo')) or\
           (duration >= @duration_criteria) 
        """
head(df.query(query).reset_index(drop=True), 5)

10 rows, 15 columns


,grouping,orders,description,second_col,qty,price,effective,expired,duration,delimitted_list,list_1,list_2,list_3,combined_listcol,X8
0,A100,23899001,First Row,Aidan Scally,14,1.23,2015-10-21,2019-12-31,1532.0,123;456;789,123,456,789,"[123, 456, 789]","=CONCATENATE(A2, B2, C2)"
1,A103,23899006,Sixth Row,Derek Hunt,532,634.23,2019-04-21,2020-05-07,382.0,alpha; bravo; charlie,alpha,bravo,charlie,"[d20, tg54, kde40]","=CONCATENATE(A3, B3, C3)"
2,A101,23899002,Second Row,Eoin Mcallister,103,4.32,2016-10-21,2020-12-31,1532.0,ss;11;33,ss,11,33,"[alpha, bravo, charlie]","=CONCATENATE(A4, B4, C4)"
3,A101,23899003,Third Row,Mike Tarpey,1,34.32,2017-10-21,2021-12-31,1532.0,A;b;C,A,b,C,"[ss, 11, 33]","=CONCATENATE(A5, B5, C5)"
4,A102,23899004,Fourth Row,Alan Denton,13,49.00,2018-10-21,2022-12-31,1532.0,cat;dog;books,cat,dog,books,"[A, b, C]","=CONCATENATE(A6, B6, C6)"


## df.loc[filter, 'from': 'to'] 
df.loc can be used to:<ol>
<li> apply a filter/selection to dataframe
<li> using that filter, select one or more columns to view and optionally (update)
</ol><p>
<a> href=https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.loc.html</a>

In [54]:
subset_cols = ['second_col', 'effective', 'expired']

# if 'second_col' is Alan Denton, show me selected columns second_col, effective and expired
df.loc[df['second_col'] == 'Alan Denton', subset_cols]

,second_col,effective,expired
3,Alan Denton,2018-10-21,2022-12-31
10,Alan Denton,2018-10-21,2022-12-31


In [55]:
# If second = 'Alan Denton', show me columns from 'orders' to 'duration'
df.loc[df['second_col'] == 'Alan Denton', 'orders':'duration']

,orders,description,second_col,qty,price,effective,expired,duration
3,23899004,Fourth Row,Alan Denton,13,49.0,2018-10-21,2022-12-31,1532.0
10,23899011,Fourth Row,Alan Denton,13,49.0,2018-10-21,2022-12-31,1532.0


# DataFrame meta information

## describe()
describe() shows basic statistical information.
.T transposes the x and y axes

In [56]:
df.describe() #.T

,orders,qty,price,duration
count,1.200000e+01,12.000000,12.000000,9.000000
mean,2.389901e+07,261.666667,134.004167,1540.666667
std,3.605551e+00,515.689535,218.971149,594.748686
min,2.389900e+07,1.000000,1.230000,382.000000
25%,2.389900e+07,13.750000,32.502500,1532.000000
50%,2.389901e+07,21.500000,47.400000,1532.000000
75%,2.389901e+07,239.500000,66.725000,1532.000000
max,2.389901e+07,1800.000000,634.230000,2760.000000


### select numeric (exclude object)

In [57]:
# Give me columns that are NOT string/text based -> numbers, floats and dates
df.select_dtypes(exclude='object').head()

,orders,description,qty,price,effective,expired,duration
0,23899001,First Row,14,1.23,2015-10-21,2019-12-31,1532.0
5,23899006,Sixth Row,532,634.23,2019-04-21,2020-05-07,382.0
1,23899002,Second Row,103,4.32,2016-10-21,2020-12-31,1532.0
2,23899003,Third Row,1,34.32,2017-10-21,2021-12-31,1532.0
3,23899004,Fourth Row,13,49.00,2018-10-21,2022-12-31,1532.0


In [58]:
# Give me columns where the column name starts with an 'o'
head(df[df.columns[df.columns.str.startswith('o')]], 3)

12 rows, 1 columns


,orders
0,23899001
5,23899006
1,23899002


In [59]:
# Give me columns that contain the letters 'a' or 'o' in the column name
head(df[df.columns[df.columns.str.contains('[ao]')]], 3)

12 rows, 6 columns


,grouping,orders,description,second_col,duration,combined_listcol
0,A100,23899001,First Row,Aidan Scally,1532.0,"[123, 456, 789]"
5,A103,23899006,Sixth Row,Derek Hunt,382.0,"[d20, tg54, kde40]"
1,A101,23899002,Second Row,Eoin Mcallister,1532.0,"[alpha, bravo, charlie]"


# Export (Excel)
## Standard Pandas -  dataframe  -> Excel workbook

In [60]:
project = 'demo'

In [61]:
file_name = f'outputs/{project} example #1 Standard pandas.xlsx'
df.to_excel(file_name, index=False, freeze_panes=(1, 0), float_format="%0.2f")
logger.info(file_name)

outputs/demo example #1 Standard pandas.xlsx


Check the output in 'outputs' folder - NOT BAD for default Pandas !
<ol>
<li>We can use a 'project name' as a prefix to the file name
<li>We can 'freeze' top row (useful!)
<li>We can even specify float format for decimal values (e.g. 2 decimals see below)

<li>We cannot specify styles easily
<li>We cannot autosize column widths (annoying)
<li>We cannot specify 'table' references (VERY useful when creating Excel formula VLOOKUPS for users)
</ol>

## Standard Pandas - multiple sheets

In [62]:
# Multiple sheets - Now we have to create an ExcelWriter object, pass a bunch of boilerplate code
# e.g. date format, we still cannot specify styles or auto-width, 

file_name = f'outputs/{project} example #2 Standard pandas multi-sheets.xlsx'

# Create a Pandas Excel writer using XlsxWriter as the engine.
with pd.ExcelWriter(file_name, engine='xlsxwriter', date_format='YYYY-MM-DD',
                    datetime_format='YYYY-MM-DD') as writer:
    df_original.to_excel(writer, sheet_name='Sheet1', index=False, freeze_panes=(1, 0))
    df.to_excel(writer, sheet_name='Sheet2', index=False, freeze_panes=(1, 0))

logger.info(file_name)

outputs/demo example #2 Standard pandas multi-sheets.xlsx


## WorkBook class - multi-sheet mode

### Single sheet

In [63]:
# Literally ONE LINE OF CODE !
WorkBook(f'outputs/{project} example #3 SaveXL single sheet', sheets=df);

Workbook: outputs/20210226_demo example #3 SaveXL single sheet.xlsx
<< mult-sheet mode >>
Sheet (range): sheet1 ($A$1:$O$13)
Completed.


### Multi-sheet

In [64]:
# Literally ONE LINE OF CODE ! -> Automatic default sheet names
file_name = f'outputs/{project} example #4 SaveXL multi sheet'
WorkBook(file_name, [df, df_original]);

Workbook: outputs/20210226_demo example #4 SaveXL multi sheet.xlsx
<< mult-sheet mode >>
Sheet (range): sheet1 ($A$1:$O$13)
Sheet (range): sheet2 ($A$1:$J$13)
Completed.


### Multi-sheet with sheet names

In [65]:
# Passing a dictionary of sheet_name: dataframe object

file_name = f'outputs/{project} example #5 SaveXL multi sheet with sheet names.xlsx'
WorkBook(file_name, {'revised': df, 'original': df_original});

Workbook: outputs/20210226_demo example #5 SaveXL multi sheet with sheet names.xlsx
<< mult-sheet mode >>
Sheet (range): revised ($A$1:$O$13)
Sheet (range): original ($A$1:$J$13)
Completed.


## WorkBook class - sheet mode

### Basic usage

In [66]:
# Create a Workbook object, pass the name and optionally you want a date/datetime prefix
wb = WorkBook(f'outputs/{project} Basic workbook object', ts_prefix=False)

# Add the sheet, passing the dataframe
wb.add_sheet(df)

# Close WorkBook object
wb.close()

Workbook: outputs/demo Basic workbook object.xlsx
<< sheet mode >>
Sheet (range): sheet1 ($A$1:$O$13)
Completed.


### Adding more sheets

In [67]:
xl_file = f'outputs/{project} example #6 WorkBook - Greater control.xlsx'
wb = WorkBook(xl_file, ts_prefix=None)

wb.add_sheet(df, sheet_name='revised data', tab_color='red', zoom=120)
wb.add_sheet(df_original, sheet_name='original', tab_color='red')

wb.close()

Workbook: outputs/demo example #6 WorkBook - Greater control.xlsx
<< sheet mode >>
Sheet (range): revised data ($A$1:$O$13)
Sheet (range): original ($A$1:$J$13)
Completed.


### Complex conditional format

In [68]:
from os.path import join

In [69]:
xl_file = f'{project} example #7 WorkBook - Multi sheet with conditional formatting.xlsx'
wb = WorkBook(join('outputs', xl_file), ts_prefix=None)
styles = wb.get_styles()

sheet_name='original'
ws = wb.add_sheet(df_original, sheet_name=sheet_name, tab_color='yellow', zoom=175)

wb.add_format(ws, column_attr={'column': 'A', 'width': 10, 'format': 'center_wrap'})
wb.add_format(ws, column_attr={'column': 'B', 'width': 11})
wb.add_format(ws, column_attr={'column': 'C', 'format': 'center', 'width': 25})

# Pass a list of dictionary formats: 
# Below, example to quickly set widths for a range of columns 
cols = ['E', 'F', 'G', 'H', 'I']
formats = [{'column': f'{c}', 'width': 10} for c in cols]

wb.add_format(ws, column_attr=formats)

sheet_name = 'revised data'
ws = wb.add_sheet(df, sheet_name, tab_color='red', zoom=175)

selected_date = datetime.strptime('2018-01-01', "%Y-%m-%d")

c= [ {'type': 'formula', 'criteria': '=$I2=2263', 'format': 'accent4'},
     {'type': 'cell', 'criteria': 'equal to', 'value': '"A103"', 'format': 'accent5', 'range': 'A'},
     {'type': 'cell', 'criteria': 'equal to', 'value': 23899003, 'format': 'accent6', 'range': 'B'} ,
     {'type': 'duplicate', 'format': 'accent1', 'range': 'C:D'},
     {'type': 'text', 'criteria': 'containing', 'value': 'Eoin', 'format': 'accent2', 'range': 'D'},
     {'type': 'data_bar', 'data_bar_2010': True, 'criteria': '=$F2>0', 'range': 'F'},
     {'type': 'date', 'criteria': 'less than', 'value': selected_date, 'format': 'accent3', 'range': 'G'},
     {'type': 'formula', 'criteria': '=$J2="cat;dog;books"', 'format': 'accent5', 'range': 'J'}]
wb.add_condition(ws, condition=c)

wb.add_condition(ws, condition={'type': '3_color_scale', 'range': 'H'})

wb.add_format(ws, column_attr={'column': 'F', 'format': 'price4'})
      
wb.close()

Workbook: outputs/demo example #7 WorkBook - Multi sheet with conditional formatting.xlsx
<< sheet mode >>
Sheet (range): original ($A$1:$J$13)
Sheet (range): revised data ($A$1:$O$13)
Completed.


# Zip / compress data

In [70]:
# Zip data with prefix of 'example' and in .xlsx -> put a date prefix on it.
zip_data(filter='example*.xlsx', ts_prefix='date', test_mode=True, mode='w', info=False)

# Zip all .csv filesa and append to zip file previously specified
zip_data(filter='*.csv', ts_prefix='date', test_mode=True, mode='a', info=False)

# Zip all files which start with demo 
zip_data(target='outputs/zip_file_for_mike', filter='demo*',
         ts_prefix='date', test_mode=False,
         info=False)

Source: outputs, filter: example*.xlsx
<<TEST MODE>> Target: outputs/20210226_zip_data.zip not created with 3 files.
Source: outputs, filter: *.csv
<<TEST MODE>> Target: outputs/20210226_zip_data.zip not created with 1 files.
Source: outputs, filter: demo*
Target: outputs/20210226_zip_file_for_mike.zip created with 5 files.


<zipfile.ZipFile [closed]>